In [ ]:
!pip install contractions
!pip install emoji
!pip install ekphrasis
!pip install -U -q PyDrive
!pip install yfinance --upgrade --no-cache-dir

import contractions
import datetime
from datetime import datetime , timedelta, date
import emoji
import itertools
import json
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import numpy as np
import os
import pandas as pd
import pprint
from pprint import pprint
import random
import re
import requests 
import seaborn as sns
import string
import sys
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import torch
import yfinance as yf
from ekphrasis.classes.segmenter import Segmenter
from pydrive.auth  import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth,  drive
from oauth2client.client import GoogleCredentials
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/HD_T_GOOG_BRK-A_TSLA_Binary_Sameday.csv')
#df = pd.read_csv('/content/drive/MyDrive/HD_T_GOOG_BRK-A_TSLA_Binary_Prevday.csv')
#df = pd.read_csv('/content/drive/MyDrive/HD_T_GOOG_BRK-A_TSLA_Percentage_Sameday.csv')
df = pd.read_csv('/content/drive/MyDrive/HD_T_GOOG_BRK-A_TSLA_Percentage_Prevday.csv')
df = df.sample(frac=1)
df.drop(df[df['Date'] <= '2019-07-20'].index, inplace = True)
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], random_state = 42, test_size = 0.1)

In [ ]:
tuned_parameters = {'vect__ngram_range': [(1, 2),(2, 2)], 'clf__C':[0.01,0.1,1], 'clf__penalty':["l1","l2"]}
text_clf = Pipeline([('vect', CountVectorizer()),('clf', LogisticRegression(max_iter=5000, solver='liblinear'))])
score = 'f1_macro'
logmodel_cv=GridSearchCV(text_clf,tuned_parameters,cv=5, scoring=score)
print()
logmodel_cv.fit(X_train.values.astype('U'), y_train)
print()
print(classification_report(y_test, logmodel_cv.predict(X_test.values.astype('U')), digits=4))
print()
logmodel_cv.best_params_
print()
logmodel_cv.best_estimator_